In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
import random

# 使用 OpenAI GPT-4o 作為主要 LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 狀態定義
class State(dict):
    question: str
    answer: str
    confidence: float
    route: str
    retries: int

# --- 分類客服 ---
classify_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "你是銀行客服人員，負責將使用者問題轉交正確部門。\n\n"
     "【客服文件】\n"
     "1. 帳務部門：處理存款、提款、餘額查詢、轉帳。\n"
     "2. 信用卡部門：處理掛失、刷卡爭議、額度調整。\n"
     "3. 若問題不屬於以上範圍，請回覆 'human'。\n\n"
     "請只輸出以下其中一個詞：'accounting'、'creditcard' 或 'human'。"),
    ("human", "{question}")
])
def classify_node(state: State):
    response = llm.invoke(classify_prompt.format_messages(question=state["question"]))
    raw = response.content.strip().lower()
    if "accounting" in raw:
        route = "accounting"
    elif "creditcard" in raw:
        route = "creditcard"
    else:
        route = "human"
    print(f"[分類] 問題: {state['question']} → 部門: {route} (重試次數={state.get('retries', 0)})")
    return {"route": route, "retries": state.get("retries", 0)}

# --- 帳務部門 ---
def accounting_node(state: State):
    ans = "您好，我是帳務部門人員，正在處理您的帳務問題。"
    print(f"[帳務部門回答] {ans}")
    return {"answer": ans}

# --- 信用卡部門 ---
def creditcard_node(state: State):
    ans = "您好，我是信用卡部門人員，正在處理您的信用卡問題。"
    print(f"[信用卡部門回答] {ans}")
    return {"answer": ans}

# --- 信心度檢查 ---
def confidence_node(state: State):
    score = round(random.uniform(0, 1), 2)
    print(f"[信心度檢查] 答案: {state['answer']} → 信心度={score:.2f}")
    return {"confidence": score}

# --- 更新 retries ---
def update_retries_node(state: State):
    retries = state["retries"] + 1
    print(f"[更新嘗試次數] retries={retries}")
    return {"retries": retries}

# --- 人工客服 ---
def human_node(state: State):
    # 判斷是直接進來，還是 retries 超過 3 次
    if state.get("retries", 0) == 0:
        ans = "AI客服無法為您解決問題，已轉人工客服處理。"
    else:
        ans = f"AI客服多次嘗試回答後仍信心不足，已轉人工客服處理。"
    
    print(f"[人工客服] {ans}")
    return {"answer": ans}

# --- LangGraph 流程 ---
graph = StateGraph(State)
graph.add_node("classify", classify_node)
graph.add_node("accounting", accounting_node)
graph.add_node("creditcard", creditcard_node)
graph.add_node("confidence", confidence_node)
graph.add_node("update_retries", update_retries_node)
graph.add_node("human", human_node)

graph.set_entry_point("classify")

# 分類決策
graph.add_conditional_edges(
    "classify",
    lambda state: state["route"],
    {
        "accounting": "accounting",
        "creditcard": "creditcard",
        "human": "human"
    }
)

# 部門 → 信心度檢查
graph.add_edge("accounting", "confidence")
graph.add_edge("creditcard", "confidence")

# 信心度決策
def decide_confidence(state: State):
    if state["confidence"] >= 0.5:   # 信心度足夠 → 結束
        return END
    elif state["retries"] < 2:       # 信心度不足，但嘗試次數 < 3
        return "update_retries"      # 先更新 retries
    else:                            # 超過嘗試次數 → 轉人工
        return "human"

graph.add_conditional_edges("confidence", decide_confidence)

# 更新 retries → 回到 classify 再跑一次
graph.add_edge("update_retries", "classify")

# 人工客服為終點
graph.add_edge("human", END)

app = graph.compile()

In [36]:
# --- 測試1 ---
print(app.invoke({"question": "請問為什麼我的轉帳失敗了？"}))

[分類] 問題: 請問為什麼我的轉帳失敗了？ → 部門: accounting (重試次數=0)
[帳務部門回答] 您好，我是帳務部門人員，正在處理您的帳務問題。
[信心度檢查] 答案: 您好，我是帳務部門人員，正在處理您的帳務問題。 → 信心度=0.32
[更新嘗試次數] retries=1
[分類] 問題: 請問為什麼我的轉帳失敗了？ → 部門: accounting (重試次數=1)
[帳務部門回答] 您好，我是帳務部門人員，正在處理您的帳務問題。
[信心度檢查] 答案: 您好，我是帳務部門人員，正在處理您的帳務問題。 → 信心度=0.92
{'question': '請問為什麼我的轉帳失敗了？', 'answer': '您好，我是帳務部門人員，正在處理您的帳務問題。', 'confidence': 0.92, 'route': 'accounting', 'retries': 1}


In [37]:
# --- 測試2 ---
print(app.invoke({"question": "我的信用卡昨天被盜刷了怎麼辦？"}))

[分類] 問題: 我的信用卡昨天被盜刷了怎麼辦？ → 部門: creditcard (重試次數=0)
[信用卡部門回答] 您好，我是信用卡部門人員，正在處理您的信用卡問題。
[信心度檢查] 答案: 您好，我是信用卡部門人員，正在處理您的信用卡問題。 → 信心度=0.02
[更新嘗試次數] retries=1
[分類] 問題: 我的信用卡昨天被盜刷了怎麼辦？ → 部門: creditcard (重試次數=1)
[信用卡部門回答] 您好，我是信用卡部門人員，正在處理您的信用卡問題。
[信心度檢查] 答案: 您好，我是信用卡部門人員，正在處理您的信用卡問題。 → 信心度=0.77
{'question': '我的信用卡昨天被盜刷了怎麼辦？', 'answer': '您好，我是信用卡部門人員，正在處理您的信用卡問題。', 'confidence': 0.77, 'route': 'creditcard', 'retries': 1}


In [38]:
# --- 測試3 ---
print(app.invoke({"question": "我想問關於房貸利率"}))

[分類] 問題: 我想問關於房貸利率 → 部門: human (重試次數=0)
[人工客服] AI客服無法為您解決問題，已轉人工客服處理。
{'question': '我想問關於房貸利率', 'answer': 'AI客服無法為您解決問題，已轉人工客服處理。', 'route': 'human', 'retries': 0}
